# Imports

In [6]:
from numpy.random import seed
seed(888)
from tensorflow import set_random_seed
set_random_seed(404)

In [7]:
import os
import numpy as np
import tensorflow as tf


In [37]:
from sklearn.model_selection import train_test_split

# Constans

In [45]:
X_TRAIN_PATH = 'MNIST/digit_xtrain.csv'
X_TEST_PATH = 'MNIST/digit_xtest.csv'
Y_TRAIN_PATH = 'MNIST/digit_ytrain.csv'
Y_TEST_PATH = 'MNIST/digit_ytest.csv'

NR_CLASSES = 10
VALIDATION_SIZE = 10000
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUTS = IMAGE_WIDTH*IMAGE_HEIGHT*CHANNELS


# Get the data

In [13]:
y_train_all = np.loadtxt(Y_TRAIN_PATH,delimiter=',',dtype=int)
y_test = np.loadtxt(Y_TEST_PATH,delimiter=',',dtype=int)

In [14]:
%%time
x_test = np.loadtxt(X_TEST_PATH,delimiter=',',dtype=int)

Wall time: 5.94 s


In [11]:
%%time
x_train_all = np.loadtxt(X_TRAIN_PATH,delimiter=',',dtype=int)

Wall time: 33.7 s


# Data preprocessing

In [15]:
# Rescale 
x_train_all,x_test = x_train_all/255.0,x_test/255.0 

# Convert target values to one-hot encoding

In [ ]:
values = y_train_all[0:5]

In [20]:
np.eye(10)

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [24]:
np.eye(10)[7]

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.])

In [31]:
y_train_all #= np.eye(NR_CLASSES)[y_train_all]


array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [28]:
y_train_all[0:5]


array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [32]:
y_test = np.eye(NR_CLASSES)[y_test]

In [33]:
y_test.shape

(10000, 10)

# Create Validation datasets from traing datasets

In [44]:
x_val = x_train_all[:VALIDATION_SIZE]
y_val = y_train_all[:VALIDATION_SIZE]

In [41]:
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

# Setup Tensorflow Graph

In [86]:
X = tf.placeholder(tf.float32,shape=[None,TOTAL_INPUTS])
Y = tf.placeholder(tf.float32,shape=[None,NR_CLASSES])

### Neural Network Architecture

#### Hyperparameters

In [87]:
nr_epochs = 5
learning_rate = 0.0001

n_hidden1 = 512
n_hidden2 = 64

# hidden layer1

In [88]:
intial_w1 = tf.truncated_normal(shape=[TOTAL_INPUTS,n_hidden1],stddev=0.1,seed=42)

In [89]:
intial_w1

<tf.Tensor 'truncated_normal_5:0' shape=(784, 512) dtype=float32>

In [90]:
w1 = tf.Variable(initial_value=intial_w1)

In [91]:
intial_b1 = tf.constant(value=0.0,shape=[n_hidden1])

b1 = tf.Variable(initial_value=intial_b1)

In [92]:
layer1_in = tf.matmul(X,w1)+b1

In [94]:
layer1_out = tf.nn.relu(layer1_in)

# hiddend layer 2

In [93]:
intial_w2 = tf.truncated_normal(shape=[n_hidden1,n_hidden2],stddev=0.1,seed=42)

In [95]:
w2 = tf.Variable(initial_value=intial_w2)

In [96]:
intial_b2 = tf.constant(value=0.0,shape=[n_hidden2])

b2 = tf.Variable(initial_value=intial_b2)

In [101]:
layer2_in = tf.matmul(layer1_out,w2) + b2

In [97]:
layer2_out = tf.nn.relu(layer2_in)

# Output layer

In [102]:
intial_w3 = tf.truncated_normal(shape=[n_hidden2,NR_CLASSES],stddev=0.1,seed=42)

w3 = tf.Variable(initial_value=intial_w3)

intial_b3 = tf.constant(value=0.0,shape=[NR_CLASSES])

b3 = tf.Variable(initial_value=intial_b3)

layer3_in = tf.matmul(layer2_out,w3) + b3

output = tf.nn.softmax(layer3_in)


# Loss, Optimisation & Metrics

### def lossfunction

In [103]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y,logits=output)) 

### def optimizer

In [99]:
optimizer = tf.train.AdamOptimizer(learning_rate)
train_step = optimizer.minimize(loss)

### Accuracy metics

In [98]:
correct_pred = tf.equal(tf.argmax(output,axis=1), tf.argmax(Y,axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))

# Runn Session

In [100]:
sess = tf.Session()

In [104]:
# Initialise all the variables
init = tf.global_variables_initializer()
sess.run(init)

In [109]:
w2.eval(sess)

array([[-0.00318106, -0.11148641, -0.09498658, ..., -0.06174564,
        -0.04453934, -0.09654885],
       [-0.07682177,  0.07720204,  0.07330777, ...,  0.05019974,
        -0.03819646, -0.09781059],
       [ 0.09191744, -0.12130337,  0.00561426, ..., -0.09149531,
        -0.02622307, -0.03810998],
       ...,
       [-0.02441056, -0.02519508,  0.07293973, ..., -0.13805985,
        -0.12919603,  0.08462437],
       [-0.10020985, -0.0341473 ,  0.06631342, ...,  0.00896058,
         0.01688146,  0.01692208],
       [ 0.04173469, -0.01844047, -0.06323416, ...,  0.11694574,
        -0.05659525, -0.1724287 ]], dtype=float32)

# Batching the data

In [108]:
size_of_batch = 1000

In [110]:
num_examples = y_train.shape[0]
nr_iterations = int(num_examples/size_of_batch)

index_in_epoch = 0


In [112]:
def next_batch(batch_size,data,labels):
    
    global num_examples
    global index_in_epoch
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    if index_in_epoch > num_examples:
        
        start = 0
        index_in_epoch = batch_size
    
    end = index_in_epoch
    
    return data[start:end],labels[start:end]
    

# Training loop

In [115]:
for epoch in range(nr_epochs):
    
    for i in range(nr_iterations):
        
        batch_x,batch_y = next_batch(batch_size=size_of_batch,data=x_train,labels=y_train)
        
        feed_dictionary = {X:batch_x,Y:batch_y}
        
        sess.run(train_step,feed_dict=feed_dictionary)
        
        batch_accuracy = sess.run(fetches=[accuracy],feed_dict=feed_dictionary)
        
    print(f'Epoch {epoch} \t | Trainig accuracy = {batch_accuracy}')

print("Done training")
        

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder' with dtype float and shape [?,784]
	 [[Node: Placeholder = Placeholder[dtype=DT_FLOAT, shape=[?,784], _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Placeholder', defined at:
  File "C:\Users\AI\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\AI\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\AI\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\AI\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\AI\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 499, in start
    self.io_loop.start()
  File "C:\Users\AI\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 132, in start
    self.asyncio_loop.run_forever()
  File "C:\Users\

In [121]:
feed_dictionary.keys()

dict_keys([<tf.Tensor 'Placeholder_2:0' shape=(?, 784) dtype=float32>, <tf.Tensor 'Placeholder_3:0' shape=(?, 10) dtype=float32>])

In [124]:
Y.shape

TensorShape([Dimension(None), Dimension(10)])